In [ ]:
import pathlib

from astropy.convolution import Gaussian2DKernel, convolve
import astropy.coordinates as coord
from astropy.io import ascii
import astropy.table as at
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from scipy.stats import binned_statistic, binned_statistic_2d
from IPython.display import HTML

# gala
import gala.coordinates as gc
import gala.dynamics as gd
import gala.potential as gp
from gala.units import galactic

from pyia import GaiaData

In [ ]:
joined = at.Table.read(
    pathlib.Path('~/data/Asteroseismology/TESS_Hon2021_GaiaEDR3.fits').expanduser(),
    hdu=1)

rave = at.Table.read(
    pathlib.Path('~/data/RAVE/rave_gaia_xmatch.fits').expanduser())
rave.rename_column('RAVE_OBS_ID', 'rave_obs_id')

rave_pars = at.Table.read(
    pathlib.Path('~/data/RAVE/dr6_gauguin_madera.fits').expanduser())
rave = at.join(rave, rave_pars, keys='rave_obs_id')

joined = at.join(joined, rave, keys='source_id', 
                 join_type='left')
joined = at.unique(joined, keys='TIC')
len(joined)

In [ ]:
edr3_actions = at.Table.read(
    pathlib.Path('~/data/GaiaEDR3/edr3-rv-good-plx-result-actions.fits').expanduser())
joined = at.join(joined, edr3_actions, 
                 keys='source_id', join_type='left')
len(joined)

In [ ]:
plt.hist(joined['mass'], bins=np.linspace(0.5, 30, 32));
plt.yscale('log')
plt.xlabel(f'mass [{u.Msun:latex_inline}]');

In [ ]:
g = GaiaData(joined)

mask = (
    np.isfinite(g.dr2_radial_velocity) &
    (g.massflag == 1) &
    (g.mass < 20*u.Msun) &
    (g.parallax_over_error > 5)
)
g = g[mask]
c = g.get_skycoord(radial_velocity=g.dr2_radial_velocity)
len(g)

Compute kinematic quantities:

In [ ]:
vcirc = 229 * u.km/u.s
galcen_frame = coord.Galactocentric()
galcen_frame

In [ ]:
galcen = c.transform_to(galcen_frame)
w0 = gd.PhaseSpacePosition(galcen.data)
L = w0.angular_momentum()
Lz = L[2]

cyl = galcen.cylindrical
vcyl = galcen.cylindrical.differentials['s']
vR = vcyl.d_rho.to_value(u.km/u.s)
vphi = (cyl.rho * vcyl.d_phi).to_value(u.km/u.s, u.dimensionless_angles())

In [ ]:
vz_z_bins = (
    np.arange(-75, 75+1e-3, 1.),
    np.arange(-1.5, 1.5+1e-3, 25/1e3)
)

In [ ]:
mass_bins = [
    (0.5, 1.4),
    (1.4, 2),
    (2, 2.6),
    (2.6, 10)
]
print([((g.mass.value > m1) & (g.mass.value < m2)).sum()
       for m1, m2 in mass_bins])

plt.hist(joined['mass'], bins=np.linspace(0.5, 10, 64));
plt.yscale('log')
plt.xlabel(f'mass [{u.Msun:latex_inline}]');
plt.vlines(np.ravel(mass_bins),
           plt.ylim()[0], plt.ylim()[1], 
           color='tab:blue')

In [ ]:
Lz_sun = 240*u.km/u.s * 8.1*u.kpc
Lz_solar = -Lz.to_value(Lz_sun)

Rg_bins = [
    [6, 8],
    [7, 9],
    [8, 10],
    [9, 12]
] * u.kpc
Lz_bins = (Rg_bins * vcirc).to_value(Lz_sun)

# Lz_bins = [
#     (0.7, 0.9), 
#     (0.9, 1.),
#     (1., 1.1),
#     (1.1, 1.2)
# ]
print([((Lz_solar > m1) & (Lz_solar < m2)).sum()
       for m1, m2 in Lz_bins])

plt.hist(Lz_solar,
         bins=np.linspace(0, 2, 32));
plt.yscale('log')
plt.xlabel('$L_z$');
plt.vlines(np.ravel(Lz_bins),
           plt.ylim()[0], plt.ylim()[1], 
           color='tab:blue')

In [ ]:
Lz_mask = np.abs(Lz_solar - 1.) < 0.15

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

ax = axes[0]
stat = binned_statistic_2d(
    galcen.v_z.to_value(u.km/u.s)[Lz_mask],
    galcen.z.to_value(u.kpc)[Lz_mask],
    g.mass.value[Lz_mask],
    statistic=np.mean,
    bins=vz_z_bins
)
H = stat.statistic.T

kernel = Gaussian2DKernel(x_stddev=0.8)
H = convolve(H, kernel)

cs = ax.pcolormesh(
    stat.x_edge, stat.y_edge, 
    H, 
    vmin=0.8, vmax=1.6,
    cmap='magma_r')

cb = fig.colorbar(cs, ax=ax)
cb.set_label('mean mass')

ax.set_xlabel(f'$v_z$ [{u.km/u.s:latex_inline}]')
ax.set_ylabel(f'$z$ [{u.kpc:latex_inline}]')

# ---

ax = axes[1]
stat = binned_statistic_2d(
    galcen.v_z.to_value(u.km/u.s)[Lz_mask],
    galcen.z.to_value(u.kpc)[Lz_mask],
    g.mass.value[Lz_mask],
    statistic=lambda x: np.nanpercentile(x, 99),
    bins=vz_z_bins
)
H = stat.statistic.T

kernel = Gaussian2DKernel(x_stddev=0.8)
H = convolve(H, kernel)

cs = ax.pcolormesh(
    stat.x_edge, stat.y_edge, 
    H, 
    # vmin=0.8, vmax=1.6,
    cmap='magma_r')

cb = fig.colorbar(cs, ax=ax)
# cb.set_label('stddev. mass')

fig.tight_layout()

In [ ]:
plt.hist((-Lz / vcirc).to_value(u.kpc), bins=np.linspace(3, 12, 64));
plt.xlabel('$R_g$ [kpc]')

In [ ]:
from astropy.stats import median_absolute_deviation as MAD

In [ ]:
ncols = len(Lz_bins)
fig, all_axes = plt.subplots(
    3, ncols, figsize=(ncols*4 + 0.5, 4 * 3),
    sharex=True, sharey=True,
    constrained_layout=True
)

for i, ((Lz1, Lz2), (Rg1, Rg2)) in enumerate(zip(Lz_bins, Rg_bins)):
    axes = all_axes[:, i]

    Lz_mask = (Lz_solar > Lz1) & (Lz_solar <= Lz2)
    Lz_ctr = 0.5 * (Lz1 + Lz2)
    axes[0].set_title(f'$R_g = {Rg1.value:.0f}$–${Rg2.value:.0f}$ kpc')
    
    vz = galcen.v_z.to_value(u.km/u.s)[Lz_mask]
    z = galcen.z.to_value(u.kpc)[Lz_mask]
    mass = g.mass.value[Lz_mask]

    stat = binned_statistic_2d(
        vz, z, mass,
        statistic=np.mean,
        bins=vz_z_bins
    )
    kernel = Gaussian2DKernel(x_stddev=1.)
    H1 = convolve(stat.statistic.T, kernel)
    
    # ---
    
#     H2, xe, ye = np.histogram2d(
#         vz, z,
#         bins=vz_z_bins,
#     )
#     sm_kernel = Gaussian2DKernel(x_stddev=2.)
#     big_kernel = Gaussian2DKernel(x_stddev=5)
#     H2 = (convolve(H2, kernel) - convolve(H2, big_kernel)) / convolve(H2, big_kernel)
    
    H2, xe, ye = np.histogram2d(
        vz, z,
        bins=vz_z_bins,
        density=True
    )

    vz_scale = 1.5 * MAD(vz)
    z_scale = 1.5 * MAD(z)
    radius = np.sqrt((vz/vz_scale)**2 + (z/z_scale)**2)
    rng = np.random.default_rng(42)
    new_phi = np.arctan2(z/z_scale, vz/vz_scale)
    rng.shuffle(new_phi)

    new_vz = vz_scale * radius * np.cos(new_phi)
    new_z = z_scale * radius * np.sin(new_phi)

    H_shuff, xe, ye = np.histogram2d(
        new_vz, new_z,
        bins=vz_z_bins,
        density=True
    )
    
    kernel = Gaussian2DKernel(x_stddev=3)
    H_shuff = convolve(H_shuff.T, kernel)
    H_diff = (convolve(H2.T, kernel) - H_shuff) / H_shuff
        
    vmin, vmax = np.nanpercentile(g.mass.value, [5, 85])
    cs = axes[0].pcolormesh(
        stat.x_edge, stat.y_edge, 
        H1, 
        vmin=vmin, vmax=vmax,
        cmap='magma_r')
    
    this_H = convolve(H2.T, Gaussian2DKernel(x_stddev=1.))
    this_H = np.log(this_H)
    vmin, vmax = np.nanpercentile(this_H.ravel(), [15, 99.9])
    cs = axes[1].pcolormesh(
        xe, ye, this_H, 
        cmap='cividis',
        vmin=vmin, vmax=vmax)
    
    cs = axes[2].pcolormesh(
        xe, ye, H_diff, 
        cmap='RdBu_r', vmin=-0.5, vmax=0.5)

# cb = fig.colorbar(cs, ax=all_axes)
# cb.set_label('mean mass')
# cb.ax.set_aspect(30)

for ax in all_axes[-1]:
    ax.set_xlabel(f'$v_z$ [{u.km/u.s:latex_inline}]')
for ax in all_axes[:, 0]:
    ax.set_ylabel(f'$z$ [{u.kpc:latex_inline}]')
    
_labels = ['mean mass', 'number density', r'$\delta n$']
for ax, label in zip(all_axes[:, -1], _labels):
    ax.set_ylabel(label)
    ax.yaxis.set_label_position('right')

In [ ]:
ncols = len(Lz_bins)
fig, all_axes = plt.subplots(
    2, ncols, figsize=(ncols*4 + 0.5, 4 * 2),
    sharex=True, sharey=True,
    constrained_layout=True
)

for i, ((Lz1, Lz2), (Rg1, Rg2)) in enumerate(zip(Lz_bins, Rg_bins)):
    axes = all_axes[:, i]

    Lz_mask = (
        (Lz_solar > Lz1) & 
        (Lz_solar <= Lz2) &
        (g.mass.value > 1.2)
    )
    Lz_ctr = 0.5 * (Lz1 + Lz2)
    axes[0].set_title(f'$R_g = {Rg1.value:.0f}$–${Rg2.value:.0f}$ kpc')
    
    vz = galcen.v_z.to_value(u.km/u.s)[Lz_mask]
    z = galcen.z.to_value(u.kpc)[Lz_mask]
    mass = g.mass.value[Lz_mask]

    stat = binned_statistic_2d(
        vz, z, mass,
        statistic=np.mean,
        bins=vz_z_bins
    )
    kernel = Gaussian2DKernel(x_stddev=1.)
    H1 = convolve(stat.statistic.T, kernel)
    
    # ---
    
#     H2, xe, ye = np.histogram2d(
#         vz, z,
#         bins=vz_z_bins,
#     )
#     sm_kernel = Gaussian2DKernel(x_stddev=2.)
#     big_kernel = Gaussian2DKernel(x_stddev=5)
#     H2 = (convolve(H2, kernel) - convolve(H2, big_kernel)) / convolve(H2, big_kernel)
    
    H2, xe, ye = np.histogram2d(
        vz, z,
        bins=vz_z_bins,
        density=True
    )

    vz_scale = 1.5 * MAD(vz)
    z_scale = 1.5 * MAD(z)
    radius = np.sqrt((vz/vz_scale)**2 + (z/z_scale)**2)
    rng = np.random.default_rng(42)
    new_phi = np.arctan2(z/z_scale, vz/vz_scale)
    rng.shuffle(new_phi)

    new_vz = vz_scale * radius * np.cos(new_phi)
    new_z = z_scale * radius * np.sin(new_phi)

    H_shuff, xe, ye = np.histogram2d(
        new_vz, new_z,
        bins=vz_z_bins,
        density=True
    )
    
    kernel = Gaussian2DKernel(x_stddev=3)
    H_shuff = convolve(H_shuff.T, kernel)
    H_diff = (convolve(H2.T, kernel) - H_shuff) / H_shuff
    
    this_H = convolve(H2.T, Gaussian2DKernel(x_stddev=1.))
    this_H = np.log(this_H)
    vmin, vmax = np.nanpercentile(this_H.ravel()[np.isfinite(this_H.ravel())], [15, 99.9])
    cs = axes[0].pcolormesh(
        xe, ye, this_H, 
        cmap='cividis',
        vmin=vmin, vmax=vmax)
    
    cs = axes[1].pcolormesh(
        xe, ye, H_diff, 
        cmap='RdBu_r', vmin=-0.5, vmax=0.5)

# cb = fig.colorbar(cs, ax=all_axes)
# cb.set_label('mean mass')
# cb.ax.set_aspect(30)

for ax in all_axes[-1]:
    ax.set_xlabel(f'$v_z$ [{u.km/u.s:latex_inline}]')
for ax in all_axes[:, 0]:
    ax.set_ylabel(f'$z$ [{u.kpc:latex_inline}]')
    
_labels = ['mean mass', 'number density', r'$\delta n$']
for ax, label in zip(all_axes[:, -1], _labels):
    ax.set_ylabel(label)
    ax.yaxis.set_label_position('right')

In [ ]:
Lz_mask = np.abs(Lz_solar - 1) < 0.2

ncols = len(mass_bins)
fig, axes = plt.subplots(1, ncols,
                         figsize=(4*ncols, 4.6),
                         sharex=True, sharey=True)
    
for i, (m1, m2) in enumerate(mass_bins):
    ax = axes.flat[i]
    
    mass_mask = (g.mass.value > m1) & (g.mass.value < m2)
    mask = mass_mask & Lz_mask
    print(mask.sum(), mass_mask.sum())

    stat = binned_statistic_2d(
        galcen.v_z.to_value(u.km/u.s)[mask],
        galcen.z.to_value(u.kpc)[mask],
        galcen.z.to_value(u.kpc)[mask],
        statistic='count',
        bins=(np.arange(-75, 75+1e-3, 1.),
              np.arange(-1.5, 1.5+1e-3, 25/1e3))
    )
    
    H = stat.statistic.T

    kernel = Gaussian2DKernel(x_stddev=0.77)
    H = convolve(H, kernel)

    cs = ax.pcolormesh(
        stat.x_edge, stat.y_edge, H, 
        cmap='magma_r',
        vmin=np.percentile(H.ravel(), 1),
        vmax=np.percentile(H.ravel(), 99)) #, vmin=-0.5, vmax=0.5)
    
    ax.axvline(np.median(galcen.v_z.to_value(u.km/u.s)[mask]), 
               zorder=100, color='#aaaaaa', alpha=0.5)
    ax.axhline(np.median(galcen.z.to_value(u.kpc)[mask]), 
               zorder=100, color='#aaaaaa', alpha=0.5)
    
#     cb = fig.colorbar(cs)
#     cb.set_label('mean mass')

    print(np.mean(galcen.v_z.to(u.km/u.s)[mask]))
    print(np.mean(galcen.z.to(u.kpc)[mask]))
    
    ax.text(-70, 1.4, 
            f'{m1:.1f}–{m2:.1f} {u.Msun:latex_inline}',
            ha='left', va='top', fontsize=15)

for ax in axes:
    ax.set_xlabel(f'$v_z$ [{u.km/u.s:latex_inline}]')

ax = axes[0]
ax.set_ylabel(f'$z$ [{u.kpc:latex_inline}]')

fig.tight_layout()

### Theta R

In [ ]:
Lz_mask = (
    (np.abs(Lz_solar - 1.) < 0.25) &
    (g.theta_R > 0) & (g.theta_R < 2*np.pi*u.rad) &
    (g.mass.value > 1.)
)

fig, ax = plt.subplots(1, 1, figsize=(7, 6))

stat = binned_statistic_2d(
    galcen.v_z.to_value(u.km/u.s)[Lz_mask],
    galcen.z.to_value(u.kpc)[Lz_mask],
    g.theta_R.value[Lz_mask],
    statistic=np.median,
    bins=vz_z_bins
)
H = stat.statistic.T

kernel = Gaussian2DKernel(x_stddev=0.8)
H = convolve(H, kernel)

cs = ax.pcolormesh(
    stat.x_edge, stat.y_edge, 
    H, 
    vmin=0, vmax=2*np.pi,
    cmap='magma_r')

cb = fig.colorbar(cs, ax=ax)
cb.set_label(r'$\theta_R$')

ax.set_xlabel(f'$v_z$ [{u.km/u.s:latex_inline}]')
ax.set_ylabel(f'$z$ [{u.kpc:latex_inline}]')

fig.tight_layout()

In [ ]:
thR_bins = np.linspace(0, 2*np.pi, 9)
thR_bins = np.array(list(zip(thR_bins[:-1], thR_bins[1:])))

ncols = len(thR_bins)
fig, axes = plt.subplots(
    2, ncols//2, figsize=(ncols//2*4 + 0.5, 4 * 2),
    sharex=True, sharey=True,
    constrained_layout=True
)

Lz1, Lz2 = ([6, 11]*u.kpc * vcirc).to_value(Lz_sun)

for i, (th1, th2) in enumerate(thR_bins):
    ax = axes.flat[i]
    mask = (
        (Lz_solar > Lz1) & 
        (Lz_solar <= Lz2) &
        (g.mass.value > 1.) &
        (g.theta_R.value > th1) &
        (g.theta_R.value <= th2)
    )
    Lz_ctr = 0.5 * (Lz1 + Lz2)
    ax.set_title(rf'$\theta_R = {th1:.1f}$–${th2:.1f}$ rad')
    
    vz = galcen.v_z.to_value(u.km/u.s)[mask]
    z = galcen.z.to_value(u.kpc)[mask]

    H2, xe, ye = np.histogram2d(
        vz, z,
        bins=vz_z_bins,
        density=True
    )
    
    this_H = convolve(H2.T, Gaussian2DKernel(x_stddev=2))
#     this_H = np.log(this_H)
    vmin, vmax = np.nanpercentile(this_H.ravel()[np.isfinite(this_H.ravel())], [15, 99.9])
    cs = ax.pcolormesh(
        xe, ye, this_H, 
        cmap='cividis',
        vmin=vmin, vmax=vmax)

In [ ]:
from matplotlib.animation import FuncAnimation

In [ ]:
nframes = 64
thR_bin_l = np.arange(0, 2*np.pi, 2*np.pi/nframes)
dthR = (25*u.deg).to_value(u.rad)
thR_bins = [(x, x+dthR) for x in thR_bin_l]
Lz1, Lz2 = ([6, 11]*u.kpc * vcirc).to_value(Lz_sun)
rng = np.random.default_rng(42)

fig, axes = plt.subplots(
    1, 2, figsize=(12, 6),
    sharex=True, sharey=True,
    constrained_layout=True
)


def get_H1_H2(i):
    th1, th2 = thR_bins[i]

    Lz_mask = (
        (Lz_solar > Lz1) & 
        (Lz_solar <= Lz2) &
        # (g.mass.value > 1.1)
        (g.mass.value > 0.1)
    )
    mask = (
        Lz_mask &
        (g.theta_R.value > th1) &
        (g.theta_R.value <= th2)
    )
    
    ax.set_title(rf'$\theta_R = {th1:.1f}$–${th2:.1f}$ rad')
    
    vz = galcen.v_z.to_value(u.km/u.s)[mask]
    z = galcen.z.to_value(u.kpc)[mask]

    H1, xe, ye = np.histogram2d(
        vz, z,
        bins=vz_z_bins,
        density=True
    )
    
    # ---
    
    vz = galcen.v_z.to_value(u.km/u.s)[Lz_mask]
    z = galcen.z.to_value(u.kpc)[Lz_mask]
    
    vz_scale = 1.5 * MAD(vz)
    z_scale = 1.5 * MAD(z)
    radius = np.sqrt((vz/vz_scale)**2 + (z/z_scale)**2)
    
    new_phi = np.arctan2(z/z_scale, vz/vz_scale)
    rng.shuffle(new_phi)

    new_vz = vz_scale * radius * np.cos(new_phi)
    new_z = z_scale * radius * np.sin(new_phi)

    H_shuff, xe, ye = np.histogram2d(
        new_vz, new_z,
        bins=vz_z_bins,
        density=True
    )
    
    kernel = Gaussian2DKernel(x_stddev=4)
    H_shuff = convolve(H_shuff.T, kernel)
    H2 = (convolve(H1.T, kernel) - H_shuff) / H_shuff
    
    H1 = convolve(H1.T, Gaussian2DKernel(x_stddev=2))
    H1 = np.log(H1)
    
    return H1, H2, (xe, ye)
    

def init():
    H1, H2, (xe, ye) = get_H1_H2(0)
    
    vmin, vmax = np.nanpercentile(H1.ravel()[np.isfinite(H1.ravel())], [15, 99.9])
    cs1 = axes[0].pcolormesh(
        xe, ye, H1, 
        cmap='cividis',
        vmin=vmin, vmax=vmax)
    
    cs2 = axes[1].pcolormesh(
        xe, ye, H2, 
        cmap='RdBu_r', vmin=-0.5, vmax=0.5)
    
    for ax in axes:
        ax.set_xlabel(f'$v_z$ [{u.km/u.s:latex_inline}]')
    axes[0].set_ylabel(f'$z$ [{u.kpc:latex_inline}]')
    
    return cs1, cs2


def func(i):
    H1, H2, (xe, ye) = get_H1_H2(i)
    
    vmin, vmax = np.nanpercentile(H1.ravel()[np.isfinite(H1.ravel())], [15, 99.9])
        
    cs1.set_array(H1.ravel())
    cs1.set_clim(vmin, vmax)
    
    cs2.set_array(H2)
    
    return cs1, cs2

cs1, cs2 = init()

anim = FuncAnimation(fig, func, frames=len(thR_bins), 
                     blit=True)

In [ ]:
HTML(anim.to_html5_video())

In [ ]:
Lz_mask = (
    (Lz_solar > 0.8) & 
    (Lz_solar <= 1.2)
)
thR_mask = (
    (g.theta_R.value > 4) &
    (g.theta_R.value < 5.)
#    (g.theta_R.value > 0) &
#    (g.theta_R.value < 2*np.pi)
)

plt.figure(figsize=(6, 6))
rad = np.sqrt(g.J_z.to_value(25*u.km/u.s * 1*u.kpc))
xx = rad * np.cos(g.theta_z)
yy = rad * np.sin(g.theta_z)
plt.hist2d(
    xx[Lz_mask & thR_mask], 
    yy[Lz_mask & thR_mask],
    bins=np.linspace(-1, 1, 81),
    norm=mpl.colors.LogNorm()
);


Jz_mask = np.abs(rad - 0.5) < 0.1
plt.figure(figsize=(6, 6))
plt.hist2d(
    g.theta_R.value[Lz_mask & Jz_mask],
    g.theta_z.value[Lz_mask & Jz_mask],
    bins=np.linspace(0, 2*np.pi, 128)
);

### RAVE

In [ ]:
plt.hist(g[Lz_mask & ~g.rave_obs_id.mask].fe_h_gauguin, 
         bins=np.linspace(-2, 1, 64));